In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("nyc_taxi_data.csv")
df

,Unnamed: 0,tpep_pickup_datetime,tpep_dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance,pickup_hrs,dropoff_hrs,day_week,tpep_pickup_timestamp,tpep_dropoff_timestamp,duration,speed
0,0,2015-01-24 18:00:28,2015-01-24 18:10:07,-73.964111,40.761398,-73.977989,40.783093,2.40,18,18,5,1422122428,1422123007,579,14
1,1,2015-01-15 4:37:29,2015-01-15 4:58:14,-73.961479,40.760410,-73.943573,40.709702,5.00,4,4,3,1421296649,1421297894,1245,14
2,2,2015-01-09 5:14:31,2015-01-09 5:47:16,-73.986893,40.761726,-73.873169,40.774326,10.25,5,5,4,1420780471,1420782436,1965,18
3,3,2015-01-29 9:52:05,2015-01-29 10:16:33,-73.965759,40.758114,-74.010399,40.701965,5.46,9,10,3,1422525125,1422526593,1468,13
4,4,2015-01-02 2:20:01,2015-01-02 2:27:37,-73.955032,40.821857,-73.950897,40.808399,1.50,2,2,4,1420165201,1420165657,456,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168147,199994,2015-01-21 20:41:07,2015-01-21 20:48:43,-73.994392,40.751530,-73.983063,40.761108,1.10,20,20,2,1421872867,1421873323,456,8
168148,199995,2015-01-17 21:18:03,2015-01-17 21:25:40,-73.960876,40.778137,-73.973915,40.764324,1.40,21,21,5,1421529483,1421529940,457,11
168149,199997,2015-01-23 11:04:12,2015-01-23 11:11:52,-73.962616,40.769730,-73.975388,40.763649,1.00,11,11,4,1422011052,1422011512,460,7
168150,199998,2015-01-01 3:00:40,2015-01-01 3:12:10,-74.007408,40.651272,-73.978722,40.678909,2.58,3,3,3,1420081240,1420081930,690,13


In [7]:
data = df.drop(columns = ['tpep_pickup_datetime','tpep_dropoff_datetime','pickup_hrs',	'dropoff_hrs',	'day_week',	'tpep_pickup_timestamp',	'tpep_dropoff_timestamp',	'duration','speed'])
data = data.drop(df.columns[0], axis =1)
data

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance
0,-73.964111,40.761398,-73.977989,40.783093,2.40
1,-73.961479,40.760410,-73.943573,40.709702,5.00
2,-73.986893,40.761726,-73.873169,40.774326,10.25
3,-73.965759,40.758114,-74.010399,40.701965,5.46
4,-73.955032,40.821857,-73.950897,40.808399,1.50
...,...,...,...,...,...
168147,-73.994392,40.751530,-73.983063,40.761108,1.10
168148,-73.960876,40.778137,-73.973915,40.764324,1.40
168149,-73.962616,40.769730,-73.975388,40.763649,1.00
168150,-74.007408,40.651272,-73.978722,40.678909,2.58


In [8]:
import math,itertools,os,csv
from numpy import matrix #moving inside function as it is not used elsewhere and is very slow.

def distance(a,b):
    '''calculates the straight line distance between two points a and b'''
    SS = sum(map(lambda x,y:(x-y)**2,a,b))
    return SS**0.5

def makecoords():
    '''generate a list of coordinates for the buckyball'''
    phi = 0.5*(1+math.sqrt(5))
    c1 = (0,1,3*phi)
    c2 = (2,(1+2*phi),phi)
    c3 = (1,2+phi,2*phi)
    combos1 = list(itertools.product((1,-1),repeat=2))
    for i in range(len(combos1)):
        combos1[i] = (1,)+combos1[i]
    combos23 = list(itertools.product((1,-1),repeat=3))
    coords = []
    for i in combos1:
        coords.append(matrix(map(lambda x,y:x*y,c1,i)).transpose()) #column vectors
    for i in combos23:
        coords.append(matrix(map(lambda x,y:x*y,c2,i)).transpose())
        coords.append(matrix(map(lambda x,y:x*y,c3,i)).transpose())
    #permutation matrices
    P1 = matrix([[0,0,1],[1,0,0],[0,1,0]])
    P2 = matrix([[0,1,0],[0,0,1],[1,0,0]])
    for i in coords[:]:
        coords.append((P1*i))
        coords.append((P2*i))
    coords = [tuple(i.transpose().tolist()[0]) for i in coords]
    return coords

def graphcoords(coords):
    '''takes a list of triples and makes a 3-D graph'''
    #moving inside function as it is not used elsewhere and is very slow.
    import pylab as p
    import mpl_toolkits.mplot3d.axes3d as p3
    (x,y,z) = zip(*coords) #unzip the list of tuples.
    fig = p.figure()
    ax=p3.Axes3D(fig)
    ax.plot_wireframe(x,y,z)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    p.show()

def makeadjmat(coords):
    '''make a 60x60 adjacency matrix for the coordinates.'''
    D = [[distance(i,j) for j in coords] for i in coords]
    for i in range(len(D)):
        for j in range(len(D[i])):
            if D[i][j] != 2.0:
                D[i][j] = 0
            else:
                D[i][j] = 1
    return D

def writecsv(matrix,ofile):
    '''write the matrix to a csv file'''
    with open(ofile,'wb') as f:
        writer = csv.writer(f)
        for row in matrix:
            writer.writerow(tuple(row))
            
if __name__ == "__main__":
    ofile = '/content/nyc_taxi_data.csv'
    coords = makecoords()
    D = makeadjmat(coords)
    writecsv(D,ofile)

ValueError: shapes (3,3) and (1,1) not aligned: 3 (dim 1) != 1 (dim 0)

In [9]:
%matplotlib inline

In [10]:
def data_limited(df):
    
    mask = (
        (data_year.pick_long.between(-75,-73).values) & 
        (data_year.pick_lat.between(40,41).values) &
        (data_year.drop_long.between(-75,-73).values) & 
        (data_year.drop_lat.between(40,41).values)
        )
    
    return(mask)

def data_limited2(df):
    
    mask = (
        (data_year.pick_long.between(-74.02,-73.92).values) & 
        (data_year.pick_lat.between(40.70,40.85).values) &
        (data_year.drop_long.between(-74.02,-73.92).values) & 
        (data_year.drop_lat.between(40.70,40.85).values)
        )
    
    return(mask)

In [12]:
mask = (data_limited2(data_year))
print('Routes : ',sum(mask))
print('Trips : ',data_year.trips[mask].sum())

NameError: name 'data_year' is not defined

In [13]:
len(data_year[mask].start.unique())

NameError: name 'data_year' is not defined

In [ ]:
import pandas as pd
import csv
from queue import PriorityQueue

# CSE 362 - Artificial Intelligence 
# Program for calculating distance between two cities & best route 
# using Uniform Cost Search algorithm 
# Ayşe Nida Dinç - 20160808047

class CityNotFoundError(Exception):
    def __init__(self, city):
        print("%s does not exist" % city)


def build_graph(path):

    file = pd.read_csv('/content/nyc_taxi_data.csv')
    # routes = {}
    next(file)
    for row in file:
        
        row = row.split(',')
        routes.setdefault(row[0], []).append((row[1],row[2]))
        routes.setdefault(row[1], []).append((row[0],row[2]))
        
    file.close()
    
   # print(routes)
    return routes


def uniform_cost_search(graph, start_city, destination_city):

    visited = set() 
    route = [] 
    priority_queue = PriorityQueue() 
    priority_queue.put((0, [start_city])) 
    
    while priority_queue:

        if priority_queue.empty(): 
            print ('distance: infinity \nroute: \nnone')
            break

        distance, route = priority_queue.get() 
        city = route[len(route)-1]

        if city not in visited:
            visited.add(city) 

            if city == destination_city: 
                route.append(distance)
                display_route(graph,route)
                return route

        childs = graph[city]
        neighbor=[i[0] for i in childs] 

        for i in neighbor:
            if i not in visited: 
            
                totaldistance = distance + int(city_to_neighbor(graph, city, i))
                temp = route[:]
                temp.append(i)
                priority_queue.put((totaldistance, temp)) 

    return priority_queue


def city_to_neighbor(graph, current, neighbor):
    index = [i[0] for i in graph[current]].index(neighbor)
    return graph[current][index][1]


def display_route(graph,route):
    length = len(route)
    distance = route[-1]
    print()
    print('Distance between cities: %s km'%(distance))
    print()
    print('Best route: ')
    count = 0
    while count < (length-2):
        km = city_to_neighbor(graph, route[count], route[count+1]) 
        print('%s -> %s %s' %(route[count],route[count+1],km))
        count+=1
    return




if __name__ == "__main__":
    
    while True:
        try:
            inputFile = input("Enter road map path: ")
            test = open(inputFile, 'r').readlines()
        except FileNotFoundError:
            print("Wrong file or file path, please try again!")
        else:
            break
       
    graph = build_graph(inputFile)

    while True:
        try:
            start_city = input("Enter the start city: ")
            if start_city not in graph: 
                raise CityNotFoundError(start_city)
            break
        except CityNotFoundError:
            print("City not found on map, try again!")
    
    while True:
        try:
            destination_city = input("Enter the destination city: ")
            if destination_city not in graph: 
                raise CityNotFoundError(destination_city)
            break
        except CityNotFoundError:
            print("City not found on map, try again!")

   
    uniform_cost_search(graph, start_city, destination_city) 
         
    pass

Enter road map path: a
Wrong file or file path, please try again!
